# Discover & Process Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import explore_products
from eo_tools.util import show_insar_coh, show_insar_phi, show_sar_int
from eo_tools.S1.snap_dbg import process_InSAR
import geopandas as gpd
import folium
import folium.plugins
import logging
import os

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-01",
    "end": "2023-09-20",
    "geom": shp
}
results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.
- The following function can be used to visually find such products 

In [ ]:
explore_products(results, shp)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [4, 5]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, outputs_prefix="/data/S1/", extract=False)

# Process

In [ ]:
# extract paths for processing
insar_dirs = process_InSAR(
    dl[0],
    dl[1],
    outputs_prefix="/data/res",
    tmp_dir="/data/tmp",
    aoi_name = "Morocco_DBG_Coreg",
    # shp=shp,
    pol="VV",
    coh_only=False,
    intensity=False,
    clear_tmp_files=False,
    resume=False,
    apply_ESD=False
)

In [ ]:
import rasterio as rio
import numpy as np
from eo_tools.S1.util import boxcar
with rio.open("/data/tmp/IW2_VV_2023-09-15-183345_2023-09-03-183344_Morocco_DBG_Coreg_coreg.dim.tif") as ds:
    # print(ds.profile)
    prm = ds.read(1) + 1j * ds.read(2)
    sec = (ds.read(3) + 1j * ds.read(4))#.conj()

box_az=3
box_rg=10
def avg_ampl(arr, box_az, box_rg):
    return np.sqrt(boxcar((arr * arr.conj()).real, box_az, box_rg))

# normalize complex coherences
pows = avg_ampl(prm, box_az, box_rg) * avg_ampl(sec, box_az, box_rg)

valid = (pows > 0) & ~np.isnan(pows)

coh = np.full_like(prm, np.nan + 1j * np.nan, dtype=prm.dtype)
coh[valid] = boxcar(prm * sec.conj(), box_az, box_rg)[valid] / pows[valid]
coh = np.abs(coh)

In [ ]:
out_dir = f"/data/res/test_orbit_no_topo_orig_orb/"
file_coh = f"{out_dir}/coh_3_10.tif"
import xarray as xr
da = xr.open_dataset(file_coh, engine="rasterio")
arr_me = da["band_data"][0]
plt.figure(figsize=(20, 10))
plt.imshow(arr_me[3500:,0:4000:8]>0.9, interpolation="none", vmin=0, vmax=1)
# plt.imshow(arr_me[5:,2:]-coh, interpolation="none")
plt.gray()
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
plt.imshow(coh[3500:,0:4000:8]>0.9, interpolation="none", vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.gray()

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow((arr_me[3:-2,2:]-coh)[3500:,0:4000:8]>0.1, interpolation="none")
plt.gray()
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
(arr_me[:-5,:-2]-coh).mean()

# Visualize outputs on a folium map

In [ ]:
show_sar_int(insar_dirs[0])

In [ ]:
show_insar_phi(insar_dirs[0])

In [ ]:
show_insar_coh(insar_dirs[0])